### ライブラリのインポート

In [7]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import os
import math
import optuna
import re
import random
import warnings

# 全ての警告を無視する
warnings.filterwarnings("ignore")

# OptunaのロギングレベルをERRORに設定
optuna.logging.set_verbosity(optuna.logging.ERROR)

### 計算用関数

In [8]:
# 行数を指定すると所用時間を返す関数
def knapsack(row_x, data_x,required_time_x, waiting_time_x, move_time_x, attraction_no_x):
    waiting_time_x = data_x.iloc[row_x, attraction_no_x]
    ans = required_time_x + waiting_time_x + move_time_x
    return ans


# 合計所要時間と合計人気度を算出する関数
def ride_calculation(data, ride_lst, required_times, popularity, move_time):
    # print(data)
    # print(ride_lst)
    # print(required_times)
    # print(move_time)
    total_popularity = 0
    total_time = 0
    ride_time = []
    new_ride_lst = []
    # start_time = data.index[0]
    # print("start time is:", calculate_time(start_time))
    row = 0
    try:
        c=0
        for i in ride_lst:
            # total_timeが675以上かつiが0または1の場合にループを強制終了
            # ソアリン、トイストーリーMは20:15以降DPA所持者のみ乗車可能
            if (data.index[row]) >= 675 and (i == 0 or i == 1):
                c+=1
                continue
            
            else:
                # iが8以上の場合、次のループへ(該当しないアトラクション)
                if i >= 8 or i < 0:
                    # print("不明なアトラクション：", i)
                    c+=1
                    continue
                elif c>0 and i==ride_lst[c-1]:
                    c+=1
                    continue
                else:
                    # print("-" * 40)
                    # print(calculate_time(data.index[row]))
                    ride_time.append(data.index[row])
                    # print(attractions.get(i))
                    # print("移動時間", move_time)
                    waiting_time = data.iloc[row, i]
                    # print("待ち時間:", waiting_time)
                    required_time = required_times[i]
                    # print("観賞時間:", required_time)
                    ans = knapsack(row, data, required_time, waiting_time, move_time, i)
                    # print("合計所要時間:", ans)
                    total_time += ans
                    row = int(np.ceil(total_time / 15))
                    total_popularity += popularity[i]
                    new_ride_lst.append(i)
                    c+=1
    except Exception as e:
        # print("全てのアトラクションを乗ることができません")
        pass

    # print("-" * 40)
    # print(total_popularity)
    return total_popularity, total_time, new_ride_lst, ride_time

### 最適化実行関数

In [9]:
#def constraints(trial):
#    return trial.user_attrs["constraints"]

def optimization(file_pathes):
    best_plan_lst = []
    file_path = file_pathes
    # 制約条件の範囲
    #max_allowed_time = 660  # パーク内にいる時間 9:00~20:00 計660分 (20時以降はファストパスのみ受け入れと仮定)
    max_allowed_time = 675  # パーク内にいる時間 9:00~20:15 計675分 (20:15以降はファストパスのみ受け入れと仮定)
            
    
    lamb=1.0
    tt_best=700
    j=0
    NN=35
    while(np.abs(tt_best-615)>60):
    #for j in range(2):
        #print(tt_best)
        total_time_list=[]
        

        # 目的関数の定義
        def objective(trial):
            # アトラクションの所要時間と人気度 or レビュー評価
            required_times = [5, 7, 2, 3, 3, 2, 23, 30]  # 所要時間
            popularity = [476, 465, 473, 472, 480, 452, 461, 478] # レビュー評価
            #popularity = [84, 69, 55, 45, 33, 28, 19, 22]  # 人気度
            move_time = 15

            # print(file_path)
            data = pd.read_csv(file_path)
            data = data.set_index('時間')

            # 乗り物の数を決定
            n_rides = trial.suggest_int('n_rides', 10, 20)  # 最大20の乗り物を仮定

            # 乗り物の定義
            ride_lst = [trial.suggest_int(f'ride_{i}', 0, 7) for i in range(n_rides)]

            # ride_calculation関数の呼び出し
            total_popularity, total_time, new_ride_lst, ride_time = ride_calculation(data, ride_lst, required_times, popularity, move_time)

            # 制約条件の範囲
            #max_allowed_time = 660  # パーク内にいる時間 9:00~20:00 計660分 (20時以降はファストパスのみ受け入れと仮定)
            max_allowed_time = 675  # パーク内にいる時間 9:00~20:15 計675分 (20:15以降はファストパスのみ受け入れと仮定)
            
            #trial.set_user_attr("constraints", [1000*(max_allowed_time - total_time)])
            #print("total_time:",total_time)
            #print("total_popularity:",total_popularity)
            total_time_list.append(total_time)
            
            # 正規表現で年月日部分を抽出
            match = re.search(r'\d{4}-\d{2}-\d{2}', file_path)
            date = match.group() if match else None
            best_plan_lst.append([trial.number, date, total_popularity, total_time, ride_lst, new_ride_lst, ride_time])

            
            return total_popularity+lamb*(max_allowed_time - total_time)
            

            # 制約を満たす場合のみ最適化対象とする
            #if total_time <= max_allowed_time:
                # 正規表現で年月日部分を抽出
                #match = re.search(r'\d{4}-\d{2}-\d{2}', file_path)
                #date = match.group() if match else None

                #best_plan_lst.append([trial.number, date, total_popularity, total_time, ride_lst, new_ride_lst, ride_time])
                # print(total_popularity)
                #return total_popularity
            #else:
                # 制約を満たさない場合はペナルティを課す
                #return total_popularity+(max_allowed_time-total_time)*100


            #####
            # 制約を満たす場合のみ最適化対象とする
            #trial.set_user_attr("constraints",[max_allowed_time - total_time])
            # 正規表現で年月日部分を抽出
            #match = re.search(r'\d{4}-\d{2}-\d{2}', file_path)
            #date = match.group() if match else None
            #best_plan_lst.append([trial.number, date, total_popularity, total_time, ride_lst, new_ride_lst, ride_time])
            # print(total_popularity)
            #return total_popularity
            #####



        # Optunaによる最適化
        study = optuna.create_study(direction="maximize")  # 人気度を最大化
        study.optimize(objective, n_trials = NN)
        best_trial = study.best_trial
        tt_best = total_time_list[best_trial.number] #最適解における所要時間の値。これをペナルティに用いる。

        best_trial = study.best_trial
        #print(f"Best trial: {best_trial.number}")
        #print(f"Best popularity: {best_trial.value}")
        #print(f"Parameters: {best_trial.params}")
        #print("total_time here is:", tt_best)
        
        
        
        lamb_list=[]
        def objective_dual(trial_dual):
            #lamb_list=[]
            # アトラクションの所要時間と人気度 or レビュー評価
            required_times = [5, 7, 2, 3, 3, 2, 23, 30]  # 所要時間
            popularity = [476, 465, 473, 472, 480, 452, 461, 478] # レビュー評価
            #popularity = [84, 69, 55, 45, 33, 28, 19, 22]  # 人気度
            move_time = 15

            #file_path = '0724/2022-07-24.csv'
            # print(file_path)
            data = pd.read_csv(file_path)
            data = data.set_index('時間')

            # 乗り物の数を決定
            n_rides = trial_dual.suggest_int('n_rides', 10, 20)  # 最大20の乗り物を仮定

            # 乗り物の定義
            ride_lst = [trial_dual.suggest_int(f'ride_{i}', 0, 7) for i in range(n_rides)]

            # ride_calculation関数の呼び出し
            total_popularity, total_time, new_ride_lst, ride_time = ride_calculation(data, ride_lst, required_times, popularity, move_time)

            # 制約条件の範囲
            #max_allowed_time = 660  # パーク内にいる時間 9:00~20:00 計660分 (20時以降はファストパスのみ受け入れと仮定)
            max_allowed_time = 675  # パーク内にいる時間 9:00~20:15 計675分 (20:15以降はファストパスのみ受け入れと仮定)
            #lamb_x = 0.8*trial_dual.suggest_int('n_rides', 1, 3)
            lamb_x = trial_dual.suggest_float('lamb_x', 0.5, 2)
            #print("lamb:",lamb_x)
            lamb_list.append(lamb_x)
            #print("total_time:",total_time)
            #print("total_popularity:",total_popularity)
            penalty=max(0,(max_allowed_time - tt_best))
            #print("penalty",penalty)
            
            
            # 正規表現で年月日部分を抽出
            #match = re.search(r'\d{4}-\d{2}-\d{2}', file_path)
            #date = match.group() if match else None
            #best_plan_lst.append([trial_dual.number, date, total_popularity, total_time, ride_lst, new_ride_lst, ride_time])
            
            
            return total_popularity+lamb_x*penalty
        
        study_dual = optuna.create_study(direction="minimize")  # 人気度を最小化
        study_dual.optimize(objective_dual, n_trials = NN)


        best_trial_dual = study_dual.best_trial
        lamb = lamb_list[best_trial_dual.number]
        j=j+1
        #print("tt_best2:",tt_best)
        
    # 出力データのトリミング        
    # DataFrameに変換
    best_plan_df = pd.DataFrame(best_plan_lst)
    
    # 4列目（インデックスでは3列目）の値がmax_allowed_timeを超える行を削除
    filtered_plan = best_plan_df[best_plan_df.iloc[:, 3] <= max_allowed_time]
    
    best_plan = filtered_plan.loc[filtered_plan.iloc[:, 2].idxmax()]

    # best_trial.numberに対応する行を抽出
    #best_plan = best_plan_df[best_plan_df[0] == best_trial_dual.number].iloc[0]
    
    #display(best_plan)
    
    return best_plan


In [ ]:
import os
import pandas as pd

# 最適化の実行
#folder_path = 'testes/'  # 入力フォルダ
#folder_path = 'july/'  # 入力フォルダ
folder_path = 'august/'  # 入力フォルダ
#folder_path = 'date_data2/'  # 入力フォルダ

output_folder = 'automation_best_result_csv'  # 出力フォルダ

# 出力フォルダが存在しない場合は作成
os.makedirs(output_folder, exist_ok=True)

file_names = sorted(os.listdir(folder_path))  # フォルダ名をソート

for file_name in file_names:
    file_path = os.path.join(folder_path, file_name)
    if file_path.endswith('.csv'):
        best_csv = []  # 各ファイル用のリストを初期化
        for i in range(10):  # ファイルにつき何回最適化を行うか
            plan = optimization(file_path)
            plan_data = plan.iloc[1:]
            best_csv.append(plan_data.values.tolist())
            # 追加のロジックをここに記述することもできます

        # best_csvリストをDataFrameに変換
        columns = ["日付", "トータル満足度", "トータル所要時間", "予定プラン", "最適プラン", "タイムスケジュール"]
        df_best_csv = pd.DataFrame(best_csv, columns=columns)

        # 日付をインデックスに設定
        df_best_csv = df_best_csv.set_index("日付")

        # CSVファイル名を生成
        base_name = os.path.splitext(file_name)[0]  # .csvを除去
        output_file_name = f're_best_results_{base_name}_r10_t35_review.csv'
        
        # 出力ファイルのパスを作成
        output_file_path = os.path.join(output_folder, output_file_name)
        
        # DataFrameをCSVファイルとして保存
        df_best_csv.to_csv(output_file_path, index=True)
        print(output_file_name,'　has been output')


re_best_results_2022-08-01_r10_t35_review.csv 　has been output
re_best_results_2022-08-02_r10_t35_review.csv 　has been output
re_best_results_2022-08-03_r10_t35_review.csv 　has been output
re_best_results_2022-08-04_r10_t35_review.csv 　has been output
re_best_results_2022-08-05_r10_t35_review.csv 　has been output
re_best_results_2022-08-06_r10_t35_review.csv 　has been output
re_best_results_2022-08-07_r10_t35_review.csv 　has been output
re_best_results_2022-08-08_r10_t35_review.csv 　has been output
re_best_results_2022-08-09_r10_t35_review.csv 　has been output
re_best_results_2022-08-10_r10_t35_review.csv 　has been output
re_best_results_2022-08-11_r10_t35_review.csv 　has been output
re_best_results_2022-08-12_r10_t35_review.csv 　has been output
re_best_results_2022-08-13_r10_t35_review.csv 　has been output
re_best_results_2022-08-14_r10_t35_review.csv 　has been output
re_best_results_2022-08-15_r10_t35_review.csv 　has been output
re_best_results_2022-08-16_r10_t35_review.csv 　has been